# 🧩 Bloque 0 — Setup inicial

En este notebook se extraen las **características de color, textura y forma**
de las imágenes de hojas de papa, que servirán como entrada para el modelo de Machine Learning.  

**Objetivo:** transformar las imágenes en vectores numéricos que describan sus propiedades visuales.


In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from tqdm import tqdm

## 🧠 Bloque 1 — Parámetros globales

En este bloque se definen las **rutas base** del proyecto y los parámetros generales que controlan el flujo de procesamiento.  
Las variables establecen:

- La carpeta donde se encuentran las imágenes ya redimensionadas (`base_path`).  
- La ruta donde se guardará el dataset resultante con las características extraídas (`output_csv`).  

Estos parámetros permiten mantener una estructura organizada y reutilizable para las siguientes etapas del análisis.


In [9]:
base_path = "../data/2_data_resize"
output_csv = "../data/3_data_extract_features/features_dataset.csv"

## 🌈 Bloque 2 — Extracción de características de color

En este bloque se calculan **estadísticas básicas de color** para cada imagen en el espacio **RGB**.  
Se obtienen dos tipos de medidas por canal (rojo, verde y azul):

- **Media (mean):** representa el color promedio de la hoja.  
- **Desviación estándar (std):** mide la variación o dispersión de los tonos dentro de la hoja.

Estas características permiten identificar **cambios de pigmentación**, **amarillamiento** o **pérdida de verdor** asociados a posibles enfermedades o deficiencias nutricionales en la planta.


In [3]:
def extract_color_features(image):
    mean_colors = np.mean(image, axis=(0, 1))
    std_colors = np.std(image, axis=(0, 1))
    return mean_colors, std_colors


## 🌀 Bloque 3 — Características de textura

En este bloque se extraen descriptores que cuantifican la **textura** de las hojas, útiles para identificar variaciones en la superficie, rugosidad o arrugas.

Se emplean dos enfoques complementarios:

- **LBP (Local Binary Patterns):** analiza microtexturas locales comparando cada píxel con sus vecinos.  
- **GLCM (Gray-Level Co-occurrence Matrix):** evalúa la distribución espacial de los niveles de gris, obteniendo medidas como:
  - **Contraste:** diferencia de intensidad entre píxeles vecinos.  
  - **Homogeneidad:** uniformidad de la textura.  
  - **Energía:** repetitividad o regularidad del patrón.  
  - **Entropía:** grado de desorden o complejidad de la textura.  

Estas métricas ayudan a distinguir **superficies sanas** de aquellas afectadas por **deterioro o deformaciones**.


In [4]:
def extract_texture_features(gray_image):
    gray = cv2.normalize(gray_image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    entropy = -np.sum(glcm * np.log2(glcm + 1e-10))
    return contrast, homogeneity, energy, entropy

## 🧾 Bloque 4 — Extracción global por carpeta

En este bloque se **recorren todas las subcarpetas del dataset**, donde cada carpeta representa una clase (por ejemplo, diferentes tipos o estados de hojas).

Para cada imagen:
1. Se **lee y convierte** a escala de grises.  
2. Se **extraen las características de color y textura** mediante las funciones definidas previamente.  
3. Se **almacenan los valores estadísticos** (promedios, desviaciones y métricas de textura) junto con su **etiqueta de clase**.

El resultado es un **DataFrame estructurado**, donde cada fila corresponde a una imagen y cada columna a una característica relevante para su posterior análisis o clasificación.


In [11]:
data_rows = []

for label in os.listdir(base_path):
    class_path = os.path.join(base_path, label)
    if not os.path.isdir(class_path):
        continue

    for img_name in tqdm(os.listdir(class_path), desc=f"Procesando {label}"):
        img_path = os.path.join(class_path, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mean_colors, std_colors = extract_color_features(image)
        contrast, homogeneity, energy, entropy = extract_texture_features(gray)

        row = {
            "mean_R": mean_colors[2],
            "mean_G": mean_colors[1],
            "mean_B": mean_colors[0],
            "std_R": std_colors[2],
            "std_G": std_colors[1],
            "std_B": std_colors[0],
            "contrast": contrast,
            "homogeneity": homogeneity,
            "energy": energy,
            "entropy": entropy,
            "label": label
        }
        data_rows.append(row)


Procesando Virus: 100%|██████████| 532/532 [00:02<00:00, 206.42it/s]


## 💾 Bloque 5 — Ejecución y guardado

En este bloque se **consolidan todas las características extraídas** en un `DataFrame` de *pandas* y se **exportan a un archivo CSV**.

Este dataset servirá como **entrada para los siguientes procesos** de análisis, visualización o clasificación con Machine Learning.

Cada fila del archivo contiene:
- Las **características de color** (medias y desviaciones por canal).  
- Las **características de textura** (contraste, homogeneidad, energía y entropía).  
- La **etiqueta** correspondiente al grupo o clase (`label`).

El CSV generado puede integrarse directamente en flujos de *data preprocessing* o modelos de aprendizaje supervisado.


In [13]:
df = pd.DataFrame(data_rows)
df.to_csv(output_csv, index=False)
print(f"Dataset guardado en: {output_csv}")
print(df.head())

Dataset guardado en: ../data/3_data_extract_features/features_dataset.csv
       mean_R      mean_G      mean_B      std_R      std_G      std_B  \
0  128.616251  139.298928  128.693559  53.584559  47.329740  67.281873   
1  120.013433  129.300203   70.988461  41.580883  40.368936  52.271149   
2  141.421616  144.346261  130.426778  39.643502  36.942884  54.940789   
3  111.892459  110.107422   80.595265  50.574446  51.036134  57.771348   
4  121.738301  127.139848   85.049904  50.031091  51.864478  55.379393   

     contrast  homogeneity    energy    entropy     label  
0  156.163657     0.202887  0.017634  12.319971  Bacteria  
1  520.175789     0.093870  0.012094  13.357643  Bacteria  
2  468.388373     0.158820  0.021977  12.430230  Bacteria  
3  796.226517     0.072948  0.009259  13.958764  Bacteria  
4  366.683176     0.159419  0.015661  12.967569  Bacteria  
